# Partition function for H in A-stars

In [ ]:
%matplotlib inline
import numpy as np
import scipy.constants as sc
from scipy.optimize import brentq
import matplotlib.pyplot as plt

In [ ]:
E0 = sc.h*sc.c*sc.Rydberg/sc.eV # Rydberg energy
a0 = sc.hbar/sc.m_e/sc.c/sc.alpha # Bohr radius

In [ ]:
def beta(T):
    return sc.eV/sc.k/T

In [ ]:
def Qn(d,T,nmax=-1):
    # density is in cm**-3: convert
    d *= 1.0e6
    if nmax < 0:
        nmax = np.int((2*a0**3*d)**(-1/6))
    bE0 = beta(T)*E0
    return np.array([4*n**2 * np.exp(-bE0*(1-1/n**2)) for n in range(1,nmax+1)])

In [ ]:
for lgd in range(13,18):
    d = 10.0**lgd
    print(lgd,np.sum(Qn(d,10000.0)))

Thus, we may take $Q = 4\exp(\beta E_0)$ for temperatures around $10^4\,\mathrm{K}$. At a density $n=10^{15}\,\mathrm{cm}^{-3}$, the probability of an H atom being in level $n=2$ is...

In [ ]:
def p2(d,T):
    bE0 = beta(T)*E0
    return 16*np.exp(-bE0*(1-1/4))/np.sum(Qn(d,T))

In [ ]:
p2(1.0e15,10000.0)

In [ ]:
def xfcn(x,d,T):
    # d in cm**-3
    d *= 1.0e6
    xi = 4.0*(sc.m_e*sc.k*T/2/np.pi/sc.hbar**2)**(3/2) * np.exp(-beta(T)*E0) / d /np.sum(Qn(d,T))
    return (1.0-x)**2 - xi*x

In [ ]:
xfcn(0.001,1.0e15,1.5e4)

In [ ]:
def xneut(d,T):
    return brentq(xfcn,0.0,1.0,args=(d,T))

In [ ]:
xneut(1.0e15,15000)

In [ ]:
d = 1.0e14
T = np.linspace(5000,20000,100)
n2frac = np.zeros_like(T)
for i in range(len(T)):
    n2frac[i] = xneut(d,T[i])*p2(d,T[i])

In [ ]:
plt.plot(T,n2frac)
plt.grid()

In [ ]:
T = 1.0e4
d=1.0e15
bE0 = beta(T)*E0
nmax = np.int(0.1*np.exp(bE0/2))
nvals = [1,2,3,nmax//10-1,nmax//10,nmax//5-1,nmax//5,nmax-1,nmax]
for n in nvals:
    print('{0:4d} & {1:4.2e} & {2:7.4f} \\\\'.format(n,n**2 * np.exp(-bE0*(1-1/n**2)),np.sum(Qn(d,T,n))))